In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Experiments: Autologging

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/autologging.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/autologging.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/experiments/autologging.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

As part of the Data science team, you may want to try different modeling approaches during experimentation phase.To guarantee reproducibility, each approach has different parameters that you need to manually track This is a time consuming task. To address this challenge, Vertex AI SDK introduces autologging, a one-line code SDK capability which leverages MLflow to provide automatic metrics and parameters tracking associated with your Vertex Experiments and Experiment Runs.

### Objective

In this tutorial, you learn how to use Vertex AI `autologging`.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Workbench
- Vertex AI Experiments

The steps performed include:

- Enable autologging in the Vertex AI SDK.
- Train Sklearn model and see the resulting experiment run with metrics and parameters autologged to Vertex AI Experiments without setting an experiment run.
- Train Tensorflow model, check autologged metrics and parameters to Vertex AI Experiments by manually setting an experiment run with `aiplatform.start_run()` and `aiplatform.end_run()`.
- Disable autologging in the Vertex AI SDK, train a Pytorch model and check that none of the parameters or metrics has been logged.


### Dataset

The dataset is the [UCI Car Evaluation dataset](https://archive-beta.ics.uci.edu/dataset/19/car+evaluation), which is derived from simple hierarchical decision model and it contains attributions to predict car evaluation class.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI Experiments
* Vertex AI Tensorboard
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook.

In [ ]:
# Install the packages
! pip3 install --user --upgrade pandas scikit-learn category_encoders tensorflow torch torchdata torchmetrics mlflow
! pip3 install --user --upgrade google-cloud-aiplatform

### Colab only: Uncomment the following cell to restart the kernel.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of length 8
def generate_uuid():
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=8))


UUID = generate_uuid()

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
from google.colab import auth

auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = "gs://your-bucket-name-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### Set up project template

Set the folder you use in this tutorial.

In [ ]:
import os

tutorial_path = os.path.join(os.getcwd(), "sdk_autologging_tutorial")
data_path = os.path.join(tutorial_path, "data")

for path in tutorial_path, data_path:
    os.makedirs(path, exist_ok=True)

### Download dataset

Download the car evaluation dataset from the official Google Cloud bucket.

In [ ]:
from urllib import request

DATA_URL = "http://cloud-samples-data.storage.googleapis.com/vertex-ai/dataset-management/datasets/uci_car_eval/car_evaluation_preprocessed.csv"
data_filepath = os.path.join(data_path, "car_evaluation_data.csv")
request.urlretrieve(DATA_URL, data_filepath)

import pandas as pd

COLUMN_NAMES = ["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"]
df = pd.read_csv(data_filepath)
df["class"] = df["class"].replace({"unacc": 0, "acc": 0, "good": 1, "vgood": 1})

processed_data_filepath = os.path.join(data_path, "car_evaluation_preprocessed.csv")
df.to_csv(processed_data_filepath, index=False)

In [ ]:
!head {processed_data_filepath} -n 5

### Import libraries

Import the Vertex AI library to log experiments in Vertex AI Experiments.

In [ ]:
from google.cloud import aiplatform as vertex_ai

### Helpers

To run experiments it is not uncommon to define experiment helpers, one per each modelling approach you plan to evaluate. Below you have the following experiment helpers:

*   `train_sklearn_model`: A helper function to train a Decision Tree model using Sklearn.
*   `train_tensorflow_model`: A helper function to train a simple model using Tensorflow.
*   `train_pytorch_model`: A helper function to train a simple neural network using PyTorch.


In [ ]:
def set_seed(seed: int):
    """
    A function to set the seed for reproducibility.
    Args:
        seed: Seed to be set
    Returns:
        None
    """
    import random

    import numpy as np
    import tensorflow as tf
    import torch

    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    torch.manual_seed(seed)


def train_sklearn_model(data_path: str, test_size: int, max_depth: int):
    """
    A function to train a Decision Tree model using sklearn.
    Args:
        data_path: Path to the data
        test_size: Size of the test set
        max_depth: Maximum depth of the Decision Tree
    Returns:
        None
    """

    # Libraries
    import pandas as pd
    from category_encoders import OrdinalEncoder
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split
    from sklearn.pipeline import Pipeline
    from sklearn.tree import DecisionTreeClassifier

    # Read data
    print("Reading data...")
    df = pd.read_csv(data_path)

    # Train, test split
    print("Generating train and test data...")
    x = df[["buying", "maint", "doors", "persons", "lug_boot", "safety"]]
    y = df[["class"]]
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=test_size, shuffle=True
    )

    # Build pipeline
    print("Building pipeline...")
    pipe = Pipeline(
        [
            ("encoder", OrdinalEncoder()),
            ("model", DecisionTreeClassifier(criterion="gini", max_depth=max_depth)),
        ]
    )

    # Train model
    print("Training model...")
    pipe.fit(x_train, y_train)

    # Evaluate model
    print("Evaluating model...")
    y_pred = pipe.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("accurancy", round(accuracy, 3))


def train_tensorflow_model(
    data_path: str, test_size: float, batch_size: int, epochs: int
):
    """
    A function to train a TF model.
    Args:
        data_path: Path to the data
        test_size: Size of the test set
        batch_size: Batch size
        epochs: Number of epochs
    Returns:
        None
    """
    # Libraries
    import tensorflow as tf

    # Variables
    dataset_size = 1729
    features_values = {
        "buying": ["vhigh", "high", "med", "low"],
        "maint": ["vhigh", "high", "med", "low"],
        "doors": ["2", "3", "4", "5more"],
        "persons": ["2", "4", "more"],
        "lug_boot": ["small", "med", "big"],
        "safety": ["low", "med", "high"],
    }

    # Helpers
    def get_input_layer(features_vocabulary):
        input_map = {}
        for cat_name, cat_values in features_vocabulary.items():
            input_map[cat_name] = tf.keras.Input(
                shape=(1,), name=cat_name, dtype="string"
            )
        return input_map

    def get_features_layer(inputs_map, features_vocabulary):
        features_map = {}
        for cat_name, cat_values in features_vocabulary.items():
            # Calculate categories
            cat_index = tf.keras.layers.StringLookup(
                vocabulary=cat_values, max_tokens=5
            )(inputs_map[cat_name])
            # Create encoding layer
            cat_layer = tf.keras.layers.CategoryEncoding(num_tokens=5)(cat_index)
            features_map[cat_name] = cat_layer
        return features_map

    # Read data
    print("Reading data...")
    car_dataset = tf.data.experimental.make_csv_dataset(
        data_path,
        column_names=[
            "buying",
            "maint",
            "doors",
            "persons",
            "lug_boot",
            "safety",
            "class",
        ],
        label_name="class",
        batch_size=batch_size,
    )

    # Generating Train, test split
    print("Generating train and test data...")
    train_size = int(0.8 * dataset_size)
    test_size = int(test_size * dataset_size)
    train_dataset = car_dataset.take(train_size)
    test_dataset = car_dataset.skip(train_size).take(test_size)

    # Build model
    print("Building model...")
    inputs_layer = get_input_layer(features_values)
    features_layer = get_features_layer(inputs_layer, features_values)
    x = tf.keras.layers.Concatenate()(features_layer.values())
    x = tf.keras.layers.Dense(10, activation="relu")(x)
    x = tf.keras.layers.Dense(5, activation="relu")(x)
    output_layer = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(inputs=inputs_layer.values(), outputs=output_layer)

    # Compile model
    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=["accuracy"],
    )

    # Fit the model
    print("Training model...")
    model.fit(
        train_dataset,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=test_dataset,
    )


def train_pytorch_model(
    data_path: str, test_size: float, batch_size: int, lr: float, epochs: int, seed: int
):

    # Libraries
    import numpy as np
    import torch
    import torch.nn as nn
    import torchmetrics
    from torch.utils.data import DataLoader
    from torchdata import datapipes

    # Variables
    seed = 8
    features_map = {
        0: {"low": 0, "med": 1, "high": 2, "vhigh": 3},
        1: {"low": 0, "med": 1, "high": 2, "vhigh": 3},
        2: {"2": 0, "3": 1, "4": 2, "5more": 3},
        3: {"2": 0, "4": 1, "more": 2},
        4: {"small": 0, "med": 1, "big": 2},
        5: {"low": 0, "med": 1, "high": 2},
    }
    dataset_length = 1729

    # Helpers
    def row_processor(r):
        for i, value in enumerate(r[:-1]):
            r[i] = features_map[i][value]
        return {
            "data": np.array(r[:-1], dtype=np.float64),
            "labels": np.array(r[-1], dtype=np.float64),
        }

    # Model definition
    class SimpleNetwork(nn.Module):
        def __init__(self):
            super().__init__()
            self.linear_relu = nn.Sequential(
                nn.Linear(6, 12, dtype=torch.float64),
                nn.ReLU(),
                nn.Linear(12, 6, dtype=torch.float64),
                nn.ReLU(),
                nn.Linear(6, 3, dtype=torch.float64),
                nn.ReLU(),
                nn.Linear(3, 1, dtype=torch.float64),
            )

        def forward(self, x):
            logits = self.linear_relu(x)
            return logits

    # Read data
    print("Reading and preparing data...")
    read_dp = datapipes.iter.FileLister(data_path)
    open_dp = datapipes.iter.FileOpener(read_dp)
    parse_dp = datapipes.iter.CSVParser(open_dp, delimiter=",", skip_lines=1)
    train_dp, test_dp = datapipes.iter.RandomSplitter(
        parse_dp,
        weights={"train": 1 - test_size, "test": test_size},
        total_length=dataset_length,
        seed=seed,
    )
    map_train_dp = datapipes.iter.Mapper(train_dp, row_processor)
    map_test_dp = datapipes.iter.Mapper(test_dp, row_processor)
    train_dataloader = DataLoader(map_train_dp, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(map_test_dp, batch_size=batch_size, shuffle=False)

    # Build model
    print("Building model...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = SimpleNetwork().to(device)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    # Train model
    print("Training model...")
    model.train()
    for t in range(epochs):
        batch = 0
        for row in iter(train_dataloader):
            features, labels = row["data"].to(device), row["labels"].to(device)
            train_predictions = model(features)
            train_prediction, _ = torch.max(train_predictions, 1)
            train_loss = loss_fn(train_prediction, labels)

            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            batch += 1
            print(f"Epoch {t + 1} - Batch {batch} - Loss {train_loss.item():.4f}")

    # Test model
    print("Evaluating model...")
    metric = torchmetrics.classification.BinaryAccuracy()
    metric_values = []
    model.eval()
    for t in range(epochs):
        batch = 0
        with torch.no_grad():
            for row in iter(test_dataloader):
                features, labels = row["data"].to(device), row["labels"].to(device)
                val_predictions = model(features)
                val_prediction, _ = torch.max(val_predictions, 1)
                metric.update(val_prediction, labels)
        accuracy = metric.compute()
        metric_values.append(accuracy)
        metric.reset()

        batch += 1
        print(f"Epoch {t + 1} - Batch {batch} - Accuracy {accuracy:.4f}")

### Initialize Vertex AI SDK for Python and set seed for reproducibility

Initialize the Vertex AI SDK for Python for your project and set seed to guarantee reproducibility.

In [ ]:
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

In [ ]:
set_seed(8)

## Model Experimentation using autologging with Vertex AI Experiments

Vertex AI Experiments Autologging allows to run experiments and autologging parameters and metrics of different ML frameworks.

After initiating an Vertex AI Experiment, enable autologging using `vertex_ai.autolog()`.

There are two ways to use Autologging:

1.   *With automatic experiment run creation*
2.   *With user experiment run creation*

With *automatic experiment run creation*, you run an experiment. Vertex AI SDK automatically creates an experiment run by logging all paramenters and metrics in Vertex AI Experiments.

With *user experiment run creation*, you create an experiment using `vertex_ai.start_run(your-experiment-run-name)` and run the experiment. Then you get access to resulting paramentes and metrics after you end the experiment run with `vertex_ai.end_run()`


#### Create an experiment for tracking training parameters and metrics

To start, initiate an experiment using the `init()` method.

Because some model types like Tensorflow and Keras result in autologging time series metrics, you need to create a Tensorboard instance.

To create a Tensorboard instance, you can use `vertex_ai.Tensorboard.create()`.


<div class="alert alert-danger">Notice that if you did not activate yet, Vertex AI TensorBoard charges a monthly fee of $300 per unique active user. Learn more about [TensorBoard overview](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview). </div>


In [ ]:
autologged_experiment_name = f"autologging-experiment-{UUID}"

In [ ]:
experiment_tensorboard = vertex_ai.Tensorboard.create()
vertex_ai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET_URI,
    experiment=autologged_experiment_name,
    experiment_tensorboard=experiment_tensorboard,
    experiment_description="autolog-experiment-with-automatic-run",
)

#### Autologging an experiment with automatic experiment run creation

In this section, Vertex AI SDK automatically creates an experiment run for you by logging all paramenters and training and post-training metrics in Vertex AI Experiments.


##### Enable autologging

First, enable autologging using `vertex_ai.autolog()` method.

After calling `vertex_ai.autolog()`, any metrics and parameters from
model training calls with supported ML frameworks will be automatically
logged to Vertex Experiments.

In [ ]:
vertex_ai.autolog()

##### Run baseline experiment

Next, define your baseline model by running a Sklearn model experiment.

In [ ]:
sklearn_config = dict(data_path=processed_data_filepath, test_size=0.2, max_depth=5)
train_sklearn_model(**sklearn_config)

##### Get the experiment results

Then, use the method `get_experiment_df()` to get the results of the experiment as a pandas dataframe.

Notice how all paramenters and metrics are logged in Vertex AI Experiments.

In particular, the `run_name` has been automatically assigned and the `accurancy_score` metrics you defined has been logged too.

In [ ]:
experiment_df = vertex_ai.get_experiment_df()
experiment_df = experiment_df.T
experiment_df

#### Autologging with user experiment run creation

As mentioned above, autologging automatically assigned running experiment to an experiment run.

But you can always initialize a run within the experiment using `start_run()`.


##### Initialize a new experiment run

Track a specific run within the experiment.

In [ ]:
autologged_manual_run_experiment_name = f"autologging-tf-experiment-{UUID}"
vertex_ai.start_run(autologged_manual_run_experiment_name)

##### Run an new experiment run

Next, run a new experiment run with Tensorflow model.

In [ ]:
tf_config = dict(
    data_path=processed_data_filepath, test_size=0.2, batch_size=5, epochs=3
)
train_tensorflow_model(**tf_config)

##### Compare experiment results

After the experiment run finishes, call `end_run()` method to complete the logging for that run and you can use `get_experiment_df()` again to get the results of the experiment.

In [ ]:
vertex_ai.end_run()

In [ ]:
experiment_df = vertex_ai.get_experiment_df()
experiment_df.T

### Running experiment without autologging

Enabling autologging is optional. You can always disable autologging using `vertex_ai.autolog(disable=True)`.

#### Disable autologging

In [ ]:
vertex_ai.autolog(disable=True)

#### Run a final experiment using Pytorch

In [ ]:
pt_config = dict(
    data_path=processed_data_filepath,
    test_size=0.2,
    batch_size=64,
    epochs=2,
    lr=0.01,
    seed=8,
)
train_pytorch_model(**pt_config)

#### Check experiment results

Use `vertex_ai.get_experiment_df()` to check that the PyTorch experiment run hasn't been recorded.

In [ ]:
experiment_df = vertex_ai.get_experiment_df()
experiment_df.T

Notice that Vertex AI SDK autologging uses MLFlow's autologging in its implementation, which support Pytorch Lightning only. Then you need to manually log metrics and models for tracking this Pytorch experiment. Check out the [Vertex AI Experiment documentation](https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments) to know more.

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial

In [ ]:
import os

# Delete experiment
experiment_list = vertex_ai.Experiment.list()
for experiment in experiment_list:
    experiment.delete()

# Delete Cloud Storage objects that were created
delete_bucket = True
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI